In [5]:
import numpy as np
import xarray as xr, gc, tqdm, os
from pathlib import Path
import datetime; regenerate= f'http://www.climsystems.com Lu.S {datetime.date.today()}'

## read the data


In [2]:
root = '/media/AsustorFour_Public/project_output/Rio_Tinto_VAR_project/Data/'
# output = '/media/AsustorFour_Public/project_output/Rio_Tinto_VAR_project/Data/'

In [13]:
vn1 = 'wndgust10m'
output_path1 = f'{root}/BARRA_wind_gust_day_num/'
os.makedirs(f'{output_path1}/', exist_ok=True)

In [39]:
for year in range(1990, 2020):
    day_num = f'{root}/day_num_{vn1}_{year}.daily.nc'
    if os.path.exists(day_num): continue

    max_wind = xr.open_mfdataset(f'{root}/BARRA_daymax/*{year}*.nc',
        engine='h5netcdf', chunks='auto', parallel=True)['max_'+vn1+'_daily']
    
    threshold = 27.78

    max_wind_above_threshold = max_wind.where (max_wind > threshold)
    max_wind_above_threshold = max_wind_above_threshold.where(np.isnan(max_wind_above_threshold),other=1)

    # print(max_wind_above_threshold)
    max_wind_above_threshold =  max_wind_above_threshold.drop("height")
    max_wind_above_threshold = (max_wind_above_threshold
    .rename({'latitude':'lat','longitude':'lon'}) 
    .resample(time='1Y', skipna=True).sum() 
    .assign_attrs(regenerate=regenerate)
    ).load()
    max_wind_above_threshold.name = 'day_num_above_thres'
    max_wind_above_threshold.astype('float32').to_netcdf(f'{output_path1}/{year}.nc')

    # da_num = (da_num
    #     .rename({'latitude':'lat','longitude':'lon'})  
    #     .assign_attrs(regenerate=regenerate)
    #     ).load()

KeyboardInterrupt: 